## Chargement des fichiers

In [1]:
import pandas as pd
import re
import requests
import os
from ipywidgets import IntProgress
from IPython.display import display
from pymarc import parse_xml_to_array
import jellyfish
from datetime import datetime
import itertools

pd.options.mode.chained_assignment = None  # Remove copy in place warning for dataframe operations


# Définition du fichier d'alignement à vérifier
input_file_path = "input/verif-auto-102024/rapport_20240926010347_alignement-défini_TEST ATC-collectivites_test-collectivites.tsv"

# Définition du fichier de sortie pour les cas suspects
output_file_root = "output/verif-auto-102024/verif-auto-collectivités-ATC"

# Faut-il exporter tous les alignements vérifiés avec une colonne identifiant les cas suspects (True),
# ou exporter ces derniers à part (False)?
output_in_place = True

# Types d'alignement à vérifier
valid_types = {'auto'}

# Types de validation à effectuer
validations = {'idref-local', 'similarité', 'lieux', 'numéros', 'années'}
# Les valeurs suivantes sont possibles:
# - 'personnes' -> les notices à comparer sont de type personnes
# - 'dates' -> compare les dates de vie (personnes uniquement) présentes dans les formes principales source et cible. PEUT être combiné avec 'idref-local'
# - 'auteur-titre' ATTENTION cette validation fait un appel d'API IdRef. Utiliser uniquement lorsque nécessaire.
# - 'idref-local' -> compare les sous-champs des notices source et cible. Nécessite l'accès à un export des notices IdRef et ATC/RNV
# - 'professions' -> compare les sous-champs $c si présents (personnes uniquement). DOIT être combiné avec 'idref-local'
# - 'similarité' -> tente de calculer un coéfficient de similarité entre les formes principales. PEUT être combiné avec 'idref-local'
# - 'lieux' -> tente de comparer les sous-champs $g si présents (congrès uniquement)
# - 'numéros' -> tente de comparer les sous-champs $n si présents (congrès uniquement)
# - 'années' -> tente de comparer les sous-champs 111$d si présents (congrès uniquement)

# Définition des fichiers exportés de IdRef et ATC/RNV pour comparaison des sous-champs. Utilisé uniquement en mode 'idref-local'
idref_set = 'collectivités'
idref_records_folder = "/Users/thomas/Documents/tmp-nobackup/bcu-rnv/idref-harvest-" + idref_set
# Plusieurs fichiers ATC/RNV peuvent être utilisés. Attention à les identifier dans le même ordre.
# Valeurs possibles pour atc_sets:
# - 'personnes'
# - 'collectivités'
# - 'congrès'
atc_sets = ['collectivités', 'congrès']
atc_records_file_paths = ["input/notices-detaillees/20241028_auth_atc_corporate_subf_a-b.tsv","input/notices-detaillees/20241028_auth_atc_meeting_subf_a-d.tsv"]

# Pour limiter le chargement des fichiers IdRef (ex. pour tester), renseigner cette variable pour ne charger que les x premier fichiers.
# Pour ne pas limiter, assigner la valeur -1
idref_records_limit = -1

# Quel caractère utiliser pour séparer les valeurs provenant de champs répétés lors de leur mapping
multifield_join_char = '|'

# Fichiers temporaires pour accélérer le traitement successif lors de la comparaison des sous-champs. Si existants, ils sont utilisés
# en mode 'idref-local' en lieu et place des notices complètes ci-dessus.
idref_records_extract = "input/idref-extrait-" + idref_set + "-comparaison.csv"

# Nom de la colonne ajoutée
output_colname = "validation auto"

# Lors des comparaisons textuelles, la distance calculée sera renseignée dans cette colonne
distance_colname = "distance validation"

# Définit la valeur de distance (rapportée à la longueur de la forme source) à partir de laquelle on considère qu'il y a une différence.
distance_limit = 0.15

# Ajoute une colonne avec des détails de comparaison, pour débogger
debug_column = True

# Définition des colonnes contenant les formes sources et cible à comparer
source_form_colname = "forme principale source"
target_form_colname = "forme principale cible"
align_type_colname = "décision d'alignement"
target_colname = "réservoir cible"
target_candidatenr_colname = "nombre de candidats"
idref_id_colname = "id cible"
atc_id_colname = "id source"

# Vérifier que tous les fichiers existent vraiment avant de continuer
all_files = []
for variable in [input_file_path, atc_records_file_paths]:
    if isinstance(variable, list):
        all_files.extend(variable)
    else:
        all_files.append(variable)

for file in all_files:
    if file and not os.path.isfile(file):
        print("ATTENTION le fichier suivant n'existe pas: ",file)

# Le fichier à vérifier est chargé dans une dataframe
df = pd.read_csv(input_file_path, sep='\t', dtype = str)
print(df.shape)

# Pour la comparaison, on retire les types d'alignements ignorés ainsi que les non-alignements
df_filtered = df[(df[align_type_colname].isin(valid_types)) & (df[target_colname] == 'idref')]
df_rest = df[(~df[align_type_colname].isin(valid_types)) | (df[target_colname] != 'idref')]
print(df_filtered.shape)
print(df_rest.shape)
display(df_filtered.head())

df_filtered.reset_index(drop=True, inplace=True)  # Drop indexes since we're not going to use them

(80325, 23)
(10138, 23)
(70187, 23)


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
37,rnv-nz-auth-atc,981023280158302851,Bibliothèque municipale (Angers),NaN,NaN,NaN,NaN,auto,1,1.0,...,029461634,Bibliothèque municipale Angers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,rnv-nz-auth-atc,981023280248402851,Verein Ernst Mach (Wien),NaN,NaN,NaN,NaN,auto,1,0.9239130434782609,...,029679389,Wiener Kreis 1922-1938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,rnv-nz-auth-atc,981023280266802851,Musée national des monuments français (Paris),NaN,NaN,NaN,NaN,auto,1,1.0,...,027559203,Musée national des monuments français Paris,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,rnv-nz-auth-atc,981023280267302851,Università di Roma. Istituto di studi bizantin...,NaN,NaN,NaN,NaN,auto,1,0.9818840579710144,...,250577860,Università di Roma Istituto di studi bizantini...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,rnv-nz-auth-atc,981023280267902851,Musée de l'Institut du monde arabe (Paris),NaN,NaN,NaN,NaN,auto,1,1.0,...,029373158,Institut du monde arabe Musée,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Chargement des fichiers auxiliaires

Si une comparaison utilisant les notices complètes IdRef et ATC/RNV est demandée, charger ces dernières dans un dataframe.

In [2]:
# Fonctions nécessaires à traiter les fichiers XML provenant d'IdRef (Marc21)
tps_debut_chargement = datetime.now()

# Chargement des fichiers individuels
def parse_marcxml_file(file_path):
    records = []
    with open(file_path, 'rb') as fh:
        marc_records = parse_xml_to_array(fh)
        for record in marc_records:
            if record != None:
                record_dict = extract_record_data(record)
                if record_dict != None:
                    records.append(record_dict)
    return records

# Extraction des champs nécessaires
def extract_record_data(record):
    field_001 = record.get('001')
    if field_001:
        if idref_set == 'personnes' :
            field_100 = record.get('100')
            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '100a': field_100.get('a') if field_100 else None,
                '100b': field_100.get('b') if field_100 else None,
                '100c': field_100.get('c') if field_100 else None,
                '100d': field_100.get('d') if field_100 else None
            }
        elif idref_set == 'collectivités':
            output_110a = []
            output_110b = []
            output_110g = []
            output_111a = []
            output_111c = []
            output_111d = []
            output_111n = []
            output_411a = []
            output_411c = []
            output_411d = []
            output_411n = []
            for field_110 in record.get_fields('110'):
                output_110a.append(multifield_join_char.join(field_110.get_subfields('a')))
                output_110b.append(multifield_join_char.join(field_110.get_subfields('b')))
                output_110g.append(multifield_join_char.join(field_110.get_subfields('g')))
            for field_111 in record.get_fields('111'):
                output_111a.append(multifield_join_char.join(field_111.get_subfields('a')))
                output_111c.append(multifield_join_char.join(field_111.get_subfields('c')))
                output_111d.append(multifield_join_char.join(field_111.get_subfields('d')))
                output_111n.append(multifield_join_char.join(field_111.get_subfields('n')))
            for field_411 in record.get_fields('411'):
                output_411a.append(multifield_join_char.join(field_411.get_subfields('a')))
                output_411c.append(multifield_join_char.join(field_411.get_subfields('c')))
                output_411d.append(multifield_join_char.join(field_411.get_subfields('d')))
                output_411n.append(multifield_join_char.join(field_411.get_subfields('n')))

            record_data = {
                'idref_id' : field_001.data.replace('(IDREF)','').strip(),
                '110a': multifield_join_char.join(output_110a) if output_110a else None,
                '110b': multifield_join_char.join(output_110b) if output_110b else None,
                '110g': multifield_join_char.join(output_110g) if output_110g else None,
                '111a': multifield_join_char.join(output_111a) if output_111a else None,
                '111c': multifield_join_char.join(output_111c) if output_111c else None,
                '111d': multifield_join_char.join(output_111d) if output_111d else None,
                '111n': multifield_join_char.join(output_111n) if output_111n else None,
                '411a': multifield_join_char.join(output_411a) if output_411a else None,
                '411c': multifield_join_char.join(output_411c) if output_411c else None,
                '411d': multifield_join_char.join(output_411d) if output_411d else None,
                '411n': multifield_join_char.join(output_411n) if output_411n else None
                }
        else:
            return None
        return record_data
    else:
        return None


# Chargement des fichiers du dossier IdRef
def load_marcxml_files_from_folder(folder_path, limit, status_display):
    all_records = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.xml'): 
            file_path = os.path.join(folder_path, file_name)
            records = parse_marcxml_file(file_path)
            all_records.extend(records)
        status_display.value += 1
        if (limit > 0 and status_display.value == limit):
            break
    
    # Convert list of records to DataFrame
    df = pd.DataFrame(all_records)
    return df

# Chargement des fichiers auxiliaires si on a choisi le type de validation 'idref-local'
if 'idref-local' in validations:
    
    # Si un fichier CSV a été fourni (l'extraction a déjà été faite), charger les notices depuis ce fichier
    if idref_records_extract and os.path.isfile(idref_records_extract):
        idRef_df = pd.read_csv(idref_records_extract, dtype = str)
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis {idref_records_extract}")
    else:
        # Sinon, on prend une grande respiration et on charge tous les fichiers IdRef dans un dataframe
        
        # Affiche une barre de progression
        if idref_records_limit > 0:
            num_files = idref_records_limit
        else:
            num_files = sum(1 for entry in os.scandir(idref_records_folder) if entry.is_file() and entry.name.endswith('.xml'))
        barre_attente_idRef = IntProgress(min=1, max=num_files, description="Chargement des fichiers IdRef: ", style={'description_width':'initial'},layout={'width':'80%'})
        display(barre_attente_idRef)
        
        idRef_df = load_marcxml_files_from_folder(idref_records_folder, idref_records_limit, barre_attente_idRef)
        
        # Une fois le chargement terminé, on écrit le résultat dans un fichier CSV
        if idref_records_extract:
            idRef_df.to_csv(idref_records_extract, encoding="UTF-8", index=False)
    
        print(f"{len(idRef_df.index)} notices IdRef chargées depuis export OAI IdRef dans {idref_records_folder}")
    
    # Utiliser l'identifiant IdRef comme index (pour accélérer les requêtes)
    idRef_df.set_index('idref_id', inplace=True)
    # Retirer les éventuels doublons
    idRef_df = idRef_df[~idRef_df.index.duplicated(keep='first')]
    
    # Chargement des fichiers ATC/RNV
    atc_df_list = []
    for file_index, atc_records_file_path in enumerate(atc_records_file_paths):
        if atc_records_file_path and os.path.isfile(atc_records_file_path):
            atc_df_toadd = pd.read_csv(atc_records_file_path, sep='\t', dtype = str)
            atc_set = atc_sets[file_index]
            # Extraire le contenu des sous-champs dans des colonnes distinctes (en enlevant les dernières virgules, si présentes)
            if atc_set == 'personnes':
                atc_df_toadd['100a'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$a ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100b'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$b ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100c'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$c ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
                atc_df_toadd['100d'] = atc_df_toadd['subfields_content_for_tag2'].str.extract(r'\$\$d ([^$]+)')[0].str.replace(r'[,\s]+$', '', regex=True)
            if atc_set == 'congrès':
                # Séparer les sous-champs de tags 110 et 111 - ne semble pas nécessaire
                #atc_df_toadd['110_subfields'] = atc_df_toadd.apply(lambda row: row['subfields_content_for_tag2'] if row['tag'] == '110' else None, axis=1)
                #atc_df_toadd['111_subfields'] = atc_df_toadd.apply(lambda row: row['subfields_content_for_tag2'] if row['tag'] == '111' else None, axis=1)
                # Regrouper par id pour éviter les doublons. Par la même, on utilise également l'identifiant comme index
                atc_df_toadd = atc_df_toadd.groupby('id').agg({'repo_id': 'first', 'subfields_content_for_tag2': 'first', 'tag':'first'})
            if atc_set == 'collectivités':
                atc_df_toadd = atc_df_toadd.groupby('id').agg({'repo_id': 'first', 'subfields_content_for_tag2': 'first'})
            atc_df_toadd['type'] = atc_set
            atc_df_list.append(atc_df_toadd)
            print(f"{len(atc_df_toadd.index)} notices ATC/RNV chargées depuis le fichier {atc_records_file_path}")
        else:
            print('Attention! Le fichier des notices ATC/RNV est manquant!')
    atc_df = pd.concat(atc_df_list)
    print(f"{len(atc_df.index)} notices ATC/RNV chargées au total.")

tps_fin_chargement = datetime.now()
print("--- Temps écoulé pour le chargement des fichiers: %s ---" % format(tps_fin_chargement - tps_debut_chargement))

442952 notices IdRef chargées depuis input/idref-extrait-collectivités-comparaison.csv
28746 notices ATC/RNV chargées depuis le fichier input/notices-detaillees/20241028_auth_atc_corporate_subf_a-b.tsv
44362 notices ATC/RNV chargées depuis le fichier input/notices-detaillees/20241028_auth_atc_meeting_subf_a-d.tsv
73108 notices ATC/RNV chargées au total.
--- Temps écoulé pour le chargement des fichiers: 0:00:01.491597 ---


## Définition des fonctions utiles

In [3]:
# Fonction de comparaison des dates
def date_compare(source_date,target_date):
    source_date = str(source_date)
    target_date = str(target_date)
    # Ignorer les points d'interrogation
    source_date = source_date.replace('?','')
    target_date = target_date.replace('?','')
    # Retirer le zéro en début de date
    if source_date.startswith('0'):
        source_date = source_date[1:0]
    if target_date.startswith('0'):
        target_date = target_date[1:0]
    #print("Source: " + source_date)
    #print("Target: " + target_date)
    # Si l'une des dates à comparer est vide, on passe
    if ((source_date == '') | (target_date == '')):
        return False
    # Si l'une des dates comporte des points, ne comparer que les chiffres entre eux
    if (('.' in source_date) | ('.' in target_date)):
        #print('Comparaison avec points: ' + source_date + ' et ' + target_date)
        for char in range(0,len(source_date)):
            if ((source_date[char] != '.') & (target_date[char] != '.') & (source_date[char] != target_date[char])):
                #print('Je pense que ' + source_date[char] + ' != ' + target_date[char])
                return True
        return False
    if (source_date != target_date):
        return True
    else:
        return False

# Expression régulière pour trouver les dates
date_pattern = r'(?:ca\.|fl\.)?([0-9\.]{4}|\?)-?([0-9\.]{4}|\?)?'

# Vérification des auteurs-titre

idref_base_url = 'https://idref.fr/'

# Fonction utile pour déterminer si un champ MARC existe
def contains_tag(data, tag_value):
    return any(record['tag'] == tag_value for record in data)

# Fonction pour extraire certains champs d'une notice IdRef
def get_idref_fields(notice,fields):
    output_fields = []
    for field in fields:
        try:
            # On compare les valeurs à elles-mêmes pour s'assurer qu'elles ne sont pas "NaN"
            if notice[field] == notice[field]:
                output_fields = output_fields + notice[field].split(multifield_join_char)
        except (KeyError, TypeError) as e:
            # Ignorer s'il manque un des champs
            pass
    return output_fields
            

# Fonctions pour comparer toutes les combinaisons de nombres
def number_compare(num1, num2):
    try:
        return abs(int(num1)-int(num2))
    except ValueError as e:
        # L'une des valeurs ne semble pas être un nombre, on l'ignore
        return None

def find_number_match(array1, array2):
    combinations = itertools.product(array1,array2)
    combined_results = [number_compare(x, y) for x, y in combinations]
    if min(combined_results) == 0:
        # En tous cas une des comparaisons a retourné zéro, donc c'est un match
        return True
    else:
        return False

# Fonction pour comparer toutes les combinaisons de dates de congrès
def find_date_match(array1, array2):
    combinations = itertools.product(array1,array2)
    # La fonction date_compare retourne False si la date correspond, donc on inverse
    combined_results = [not(date_compare(x, y)) for x, y in combinations]
    if min(combined_results) == 0:
        # En tous cas une des comparaisons a été positive, donc c'est un match
        return True
    else:
        return False

# Fonction pour calculer la distance minimale sur toutes les combinaisons de termes
def find_minimal_distance(array1, array2):
    # On calcule la distance pour chaque combinaison, en ne conservant que la partie avant la virgule (en général plus significative)
    combinations = itertools.product([text.split(',')[0].strip() for text in array1], [text.split(',')[0].strip() for text in array2])
    combined_results = [jellyfish.levenshtein_distance(x, y)/len(x) for x, y in combinations]
    return min(combined_results)

## Moulinette de validation
C'est là que tout se passe!

In [8]:
# Affiche une barre de progression
numrows = df_filtered.shape[0]
barre_attente = IntProgress(min=1, max=numrows, description="État d'avancement: ", style={'description_width':'initial'},layout={'width':'80%'})
display(barre_attente)
update_interval = 100 # Ne pas rafraîchir la barre trop vite sinon ça bugge l'affichage

if not(output_in_place):
    wrong_dates = []
    wrong_types = []
    wrong_forms = []
    misc_errors = []

nb_api_calls = 0

tps_debut_moulinette = datetime.now()

for index, row in df_filtered.iterrows():
    if not ((row[source_form_colname] == '') | (row[target_form_colname] == '') | pd.isnull(row[source_form_colname]) | pd.isnull(row[target_form_colname])):

        source_dates = ''
        target_dates = ''
        verif_dates = False

        source_term = ''
        target_term = ''

        notice_idref = None
        notice_atc = None

        distance = None
        valid_score = 0

        debug_output = ''
        source_type = ''
        
        # Procéder à la validation en utilisant les notices complètes IdRef et ATC/RNV si demandé
        
        if 'idref-local' in validations:
            # La validation ne peut se faire que si les notices complètes existent
            try:
                notice_idref = idRef_df.loc[row[idref_id_colname]]
            except (KeyError, TypeError) as e:
                # Ignorer cette ligne s'il manque une des données requises
                pass
            try:
                notice_atc = atc_df.loc[row[atc_id_colname]]
                source_type = notice_atc['type']
            except (KeyError, TypeError) as e:
                # Ignorer cette ligne s'il manque une des données requises
                pass
            if 'dates' in validations:
                # Extraction des dates de vie pour les personnes
                try:
                    source_dates = re.findall(date_pattern, notice_atc['100d'])
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
                try:
                    target_dates = re.findall(date_pattern, notice_idref['100d'])
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
            if ('similarité' in validations) and (idref_set == 'personnes'):
                # Extraction des formes pour la comparaison de similarité, pour les personnes
                try:
                    source_term = notice_atc['100a']
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
                try:
                    target_term = notice_idref['100a']
                except (KeyError, TypeError) as e:
                    # Ignorer cette ligne s'il manque une des données requises
                    pass
            if 'professions' in validations:
                # Comparaison des "professions", pour les personnes
                try:
                    source_prof = notice_atc['100c']
                    target_prof = notice_idref['100c']
                    distance_prof = jellyfish.levenshtein_distance(source_prof,target_prof)/len(source_prof)
                    if distance_prof > distance_limit:
                        if (output_in_place):
                            df_filtered.loc[index, output_colname] = "Vérifier professions"
                            df_filtered.loc[index, distance_colname] = distance_prof
                    elif (output_in_place):
                            df_filtered.loc[index, output_colname] = "OK ($$c)"
                except (KeyError, TypeError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass
            if 'lieux' in validations:
                # Comparaison de type lieux
                try:
                    if source_type == 'congrès':
                        # Cas d'un congrès
                        source_locations = re.findall(r'\$\$c ([^$^)]+)',notice_atc['subfields_content_for_tag2'])
                        target_locations = get_idref_fields(notice_idref, ['111c', '411c'])
                        if (len(source_locations)) > 0 and (len(target_locations) > 0):
                            if find_minimal_distance(source_locations,target_locations) <= distance_limit:
                                # En tous cas une des comparaisons de lieux semble indiquer un match, on donne un bon score
                                valid_score += 10
                                if debug_column:
                                    debug_output = debug_output + f"Match sur lieu de congrès | "
                except (KeyError, TypeError, ValueError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass
            if 'numéros' in validations:
                # Comparaison de numéro de congrès
                try:
                    if source_type == 'congrès':
                        source_congressnr = re.findall(r'\$\$n \(?(\d+)',notice_atc['subfields_content_for_tag2'])
                        target_congressnr = get_idref_fields(notice_idref, ['111n', '411n'])
                        if (len(source_congressnr)) > 0 and (len(target_congressnr) > 0):
                            if find_number_match(source_congressnr,target_congressnr):
                                # En tous cas une des comparaisons de numéros semble indiquer un match, on donne un bon score
                                valid_score += 10
                                if debug_column:
                                    debug_output = debug_output + f"Match sur numéro de congrès | "
                except (KeyError, TypeError, ValueError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass     
            if 'années' in validations:
                # Comparaison de date de congrès
                try:
                    if source_type == 'congrès':
                        source_congressdate = re.findall(r'\$\$d \(?(\d+)',notice_atc['subfields_content_for_tag2'])
                        target_congressdate = get_idref_fields(notice_idref, ['111d', '411d'])
                        if (len(source_congressdate)) > 0 and (len(target_congressdate) > 0):
                            if find_number_match(source_congressdate,target_congressdate):
                                # En tous cas une des comparaisons de numéros semble indiquer un match, on donne un bon score
                                valid_score += 10
                                if debug_column:
                                    debug_output = debug_output + f"Match sur date de congrès | "
                except (KeyError, TypeError, ValueError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass  
            if 'similarité' in validations and (idref_set != 'personnes'):
                # Comparaison de forme principale pour collectivités et congrès
                try:
                    if source_type == 'congrès':
                        source_congressterms = re.findall(r'\$\$a ([^$]+)',notice_atc['subfields_content_for_tag2'])
                        target_congressterms = get_idref_fields(notice_idref, ['110a', '110b', '111a', '411a'])
                        if (len(source_congressterms)) > 0 and (len(target_congressterms) > 0):
                            if find_minimal_distance(source_congressterms,target_congressterms) <= distance_limit:
                                # En tous cas une des comparaisons de lieux semble indiquer un match, on donne un bon score
                                valid_score += 10
                                if debug_column:
                                    debug_output = debug_output + f"Match sur titre de congrès | "
                    elif source_type == 'collectivités':
                        source_foundterms = re.findall(r'\$\$a ([^$]+)\$\$b ([^$]+)$',notice_atc['subfields_content_for_tag2'])
                        source_prefterms = [item for match in source_foundterms for item in match]
                        target_prefterms = get_idref_fields(notice_idref, ['110a', '110b'])
                        if (len(source_prefterms)) > 0 and (len(target_prefterms) > 0):
                            min_distance = find_minimal_distance(source_prefterms,target_prefterms)
                            if min_distance <= distance_limit:
                                # En tous cas une des comparaisons de termes semble indiquer un match, on considère que c'est correct
                                if (output_in_place):
                                    df_filtered.loc[index, output_colname] = "OK (levenshtein)"
                                    df_filtered.loc[index, distance_colname] = min_distance
                                if debug_column:
                                    debug_output = debug_output + f"Match sur une des formes | "
                            else:
                                # Aucune comparaison n'est concluante, on considère qu'il faut vérifier
                                if (output_in_place):
                                    df_filtered.loc[index, output_colname] = "Vérifier forme principale"
                                    df_filtered.loc[index, distance_colname] = min_distance
                except (KeyError, TypeError, ValueError) as e:
                    # Ignorer cette validation s'il manque une des données requises
                    pass  

        # Pour les congrès, si au moins une des validations est OK, on considère que c'est correct
        if (source_type == 'congrès') and (valid_score > 10):
            if (output_in_place):
                df_filtered.loc[index, output_colname] = f"OK (congrès score {str(valid_score)})"
        if (source_type == 'congrès') and (valid_score < 10):
            if (output_in_place):
                df_filtered.loc[index, output_colname] = "Vérifier congrès"

        
        # Les comparaisons suivantes ne sont utiles que pour les notices de type "personnes"
        if 'personnes' in validations:
            # Extraire les dates de la forme principale si on ne les a pas trouvées dans les sous-champs'
            if len(source_dates) < 1:
                source_dates = re.findall(date_pattern, row[source_form_colname])
            if len(target_dates) < 1:
                target_dates = re.findall(date_pattern, row[target_form_colname])
    
            if debug_column:
                debug_output = debug_output + f"Dates {source_dates} et {target_dates} | "
            
            # Validation de date si elles sont présentes des deux côtés et si demandé
            if ((len(source_dates) > 0) and (len(target_dates) > 0) and ('dates' in validations)):
                if (date_compare(source_dates[0][0],target_dates[0][0]) | date_compare(source_dates[0][1],target_dates[0][1])):
                    # Cas potentiel de dates qui ne correspondent pas
                    if (output_in_place):
                        df_filtered.loc[index, output_colname] = "Vérifier dates"
                    else:
                        wrong_dates.append(row)
                else:
                    # Les dates semblent correspondre, utiliser cette information pour valider l'alignement
                    verif_dates = True
                    df_filtered.loc[index, output_colname] = "OK (dates)"
    
            
            # Utiliser les formes simples pour la similarité textuelle (sans chiffres et signes de ponctuation) si on n'a pas pu extraire le sous-champ $a
            if len(source_term) < 1:
                source_term = re.sub(r'[\d,-.\?()]', '', row[source_form_colname]).strip()
    
            if len(target_term) < 1:
                target_term = re.sub(r'[\d,-.\?()]', '', row[target_form_colname]).strip()
    
            if debug_column:
                debug_output = debug_output + f"Comparaison entre {source_term} et {target_term} | "
            
            # Calculer la similarité textuelle si demandé
            if (len(source_term) > 0) and (len(target_term) > 0) and ('similarité' in validations):
                distance = jellyfish.levenshtein_distance(source_term,target_term)/len(source_term)
                if debug_column:
                    debug_output = debug_output + f"distance: {str(distance)} | "
    
            if (distance is not None and distance > distance_limit and not(verif_dates)):
                # Signaler comme erreur potentielle sauf si validé par les dates
                if (output_in_place):
                    df_filtered.loc[index, output_colname] = "Vérifier forme principale"
                    df_filtered.loc[index, distance_colname] = distance
                else:
                    wrong_forms.append(row)
            elif (distance is not None and distance <= distance_limit and not(verif_dates) and output_in_place):
                    df_filtered.loc[index, output_colname] = "OK (levenshtein)"
                    df_filtered.loc[index, distance_colname] = distance

        
        # Vérifier si la notice IdRef vers laquelle on aligne est de type auteur-titre, seulement si demandé
        if 'auteur-titre' in validations:
            if notice_idref is None:
                # Si la notice cible n'est pas disponible localement, il ne s'agit probablement pas d'une notice de type auteur
                # Faire un appel d'API pour en avoir le coeur net.
                # ATTENTION cette étape est très coûteuse en temps
                idref_id = row[idref_id_colname]
                url = idref_base_url + str(idref_id) + '.json'
                #print("Appel d'API pour " + str(idref_id))
                nb_api_calls += 1
                # Appel d'API
                idref_result = requests.get(url)
                if (idref_result.status_code == 200):
                    # Identifier si le résultat contient un champ 240
                    if contains_tag(idref_result.json()['record']['datafield'], 240):
                        if (output_in_place):
                            df_filtered.loc[index, output_colname] = "Cible de type auteur-titre"
                        else:
                            wrong_types.append(row)
                else:
                    # Erreur de requête API
                    if (output_in_place):
                        df_filtered.loc[index, output_colname] = "Erreur de requête API IdRef. Code:" + str(idref_result.status_code)
                    else:
                        misc_errors.append(row)
    
    else:
        # Il manque un des champs à comparer, autre erreur
        if (output_in_place):
            df_filtered.loc[index, output_colname] = "Erreur de validation"
        else:
            misc_errors.append(row)
    
    if debug_column:
        df_filtered.loc[index, "debug"] = debug_output

    if index % update_interval == 0:
        barre_attente.value = index

display(df_filtered.head())

tps_fin_moulinette = datetime.now()
print("Nombre d'appels d'API: " + str(nb_api_calls) + " (" + str(round(nb_api_calls/numrows*100)) + "% des alignements)")
print("--- Temps écoulé pour la comparaison des alignements: %s ---" % format(tps_fin_moulinette - tps_debut_moulinette))

IntProgress(value=1, description="État d'avancement: ", layout=Layout(width='80%'), max=10138, min=1, style=Pr…

,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3,debug,validation auto,distance validation
0,rnv-nz-auth-atc,981023280158302851,Bibliothèque municipale (Angers),NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
1,rnv-nz-auth-atc,981023280248402851,Verein Ernst Mach (Wien),NaN,NaN,NaN,NaN,auto,1,0.9239130434782609,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
2,rnv-nz-auth-atc,981023280266802851,Musée national des monuments français (Paris),NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN
3,rnv-nz-auth-atc,981023280267302851,Università di Roma. Istituto di studi bizantin...,NaN,NaN,NaN,NaN,auto,1,0.9818840579710144,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Match sur une des formes |,OK (levenshtein),0.0
4,rnv-nz-auth-atc,981023280267902851,Musée de l'Institut du monde arabe (Paris),NaN,NaN,NaN,NaN,auto,1,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN


Nombre d'appels d'API: 0 (0% des alignements)
--- Temps écoulé pour la comparaison des alignements: 0:00:03.070173 ---


## Exports des fichiers de résultats

In [9]:
tps_debut_sauvegarde = datetime.now()

if not(output_in_place):
    wrong_dates_df = pd.DataFrame(wrong_dates)
    wrong_forms_df = pd.DataFrame(wrong_forms)
    misc_errors_df = pd.DataFrame(misc_errors)

    wrong_dates_df.to_excel(output_file_root + "_dates.xlsx",index=False)
    wrong_forms_df.to_excel(output_file_root + "_formes.xlsx",index=False)
    misc_errors_df.to_excel(output_file_root + "_erreurs.xlsx",index=False)
else:
    print(df_filtered.shape)
    # Combiner le fichier filtré qui a été validé avec les alignements retirés au début
    df_output = pd.concat([df_filtered,df_rest])
    print(df_output.shape)
    df_output.to_excel(output_file_root + "_tout.xlsx",index=False)

tps_fin_sauvegarde = datetime.now()
print("--- Temps écoulé pour sauvegarder les fichiers: %s ---" % format(tps_fin_sauvegarde - tps_debut_sauvegarde))
print("--- Temps total écoulé: %s ---" % format(tps_fin_sauvegarde - tps_debut_chargement))

(10138, 26)
(80325, 26)
--- Temps écoulé pour sauvegarder les fichiers: 0:00:36.819741 ---
--- Temps total écoulé: 0:13:03.354703 ---
